In [1]:
def create_text(c):
    a = c['activities']
    titre = f"Le nom du cours est {c['title']}. "
    code = f"Le code du cours est {c['code']}. "
    fac = f"Le cours est dispensé par la {c['facultyLabel']}. "
    niveau = f"Ce cours est de niveau {c['studyLevel']}. "
    credit = f"Ce cours vaut {c['credits']} crédits. " if c['credits'] != None else ''
    cours = [{} for _ in range(len(a))]
    objectif = f"L\'objectif de ce cours est: {a[0]['objective']}. " if a[0]['objective'] != None else ''
    description = f"La description de ce cours est la suivante: {a[0]['description']} " if a[0]['description'] != None else 'Ce cours n\'a pas de description'
    evaluation = f"Le type d\'évaluation de ce cours est {a[0]['evaluationComment'] if a[0]['evaluationComment'] != None else a[0]['evaluation']} " if a[0]['evaluation'] != None else ''
    duration = f"La durée de ce cours est de {a[0]['duration']}. Pendant la période {a[0]['periodicity']}. "
    activites = f"Ce cours est composé de "
    langue = f"La langue utilisé pour ce cours est {a[0]['language']}. "
    recommande = f"Les cours recommandé pour ce cours sont: {a[0]['recommended']}. " if a[0]['recommended'] != None else ''
    bibliographie = f"La bibliographie pour ce cours est {a[0]['bibliography']} "
    metaProf = ''
    cours_time = ''
    seminaire_time = ''
    tp_time = ''
    studyplan = ''

    for plan in c['listStudyPlan']:
        studyplan += f"Ce cours est compris dans le plan d\'étude {plan['studyPlanLabel']} pour {plan['planCredits']} crédits. "

    professeur = [[] for _ in range(len(a))]
    for i, b in enumerate(a):
        prof = ''
        
        for j, p in enumerate(b['activityTeachers']):
            if len(p['contactInfos']) > 0:
                if i == 0:
                    metaProf += f"{p['contactInfos'][0]['title']} {p['displayFirstName']} {p['displayLastName']}"
                    if j < len(b['activityTeachers']) - 1: metaProf += ', '
                prof += f"{p['contactInfos'][0]['title']} {p['displayFirstName']} {p['displayLastName']}"
                prof += " et par " if len(b['activityTeachers']) - 1 > j else ', ' if len(a) - 1 > i else ''
                if len(b['activityTeachers']) - 1 == j: professeur[i] = prof

        # print(b['type'])
        if b['type'] == 'Cours' and len(b['courses']) > 0: cours_time = f"{b['courses'][0]['day']}: {b['courses'][0]['startHour']} - {b['courses'][0]['endHour']}"
        if b['type'] == 'Exercices' and len(b['courses']) > 0: seminaire_time = f"{b['courses'][0]['day']}: {b['courses'][0]['startHour']} - {b['courses'][0]['endHour']}"
        if b['type'] == 'Travaux pratiques' and len(b['courses']) > 0: tp_time = f"{b['courses'][0]['day']}: {b['courses'][0]['startHour']} - {b['courses'][0]['endHour']}"
        activites += f"{b['duration'] if b['duration'] != None else ''} de {b['type'] if b['type'] != None else ''}" + f"le {b['courses'][0]['day']} de {b['courses'][0]['startHour']} à {b['courses'][0]['endHour']} dans la salle {b['courses'][0]['room']} à {b['courses'][0]['building']}" if len(b['courses']) > 0 else ''
        activites += f" dispensée par {professeur[i]}"
        activites += ' et de ' if len(a) - 1 > i else '. '
    base_url = "https://pgc.unige.ch/main/api/teachings/"
    return {
            "content": titre + code + fac + niveau + credit + objectif + description + evaluation + duration + activites + langue + recommande + bibliographie + studyplan,
            "metadata": {
                "titre": c['title'],
                "code": c['code'],
                "url": base_url + c['code'],
                "professeurs": metaProf,
                "langue": a[0]['language'],
                "periode": a[0]['periodicity'],
                "fac": c['facultyLabel'],
                "horaire_seminaire": seminaire_time,
                "horaire_cours": cours_time,
                "horaire_tp": tp_time
                
            }
            }


In [2]:
import requests

url = "https://pgc.unige.ch/main/api/teachings/"
headers = {
    "Accept": "application/json",
    "User-Agent": "Your User Agent"
}

medecine_id = [['213P006', ' 213P007', '213P009'], ['223A002', '223A003', '223A004', '223A006', '223A007', 'D200011', '223A008', '223A015', '223TP001', '223C100', '223D100', '223M001', '223IP001', '223O002', '223S002'], ['223A010', '223A011', '223A012', '223C300', '223D300', '223A016', '223P100', '223O001', '223O002', '223O003', '223O004']]
ssn_id = ['D400001', 'D400004', 'D400007', 'D400008', 'D400005', 'D400012', 'D400013', 'D400014', 'D400017', 'D400009', 'D400006', '75303', '75304', '75307', '75308', 'D400002', 'T410289', '14X032', 'D400029', 'D400009', '14X028', 'S403011', '13X011', '14X026', '14X011', '10S003', 'D400015', 'D400010', '13X005', '14X015', 'T406090', '14E182', 'T406034', 'D400016', '14X032', 'D400024', '14X021', 'D400018', 'T410289', '14X016', 'D400019', 'D400020', 'D400022', 'S403011', '14X040', 'S402002', 'T410289', 'D400018', '14X021', '14X040', 'T412020', 'D400026', 'S401121', 'S401003', 'S401109', 'T421211', 'T410012']       


def scrap_courses(ids):
    courses_content = []
    parsed_data = []
    for id in ids:
        response = requests.get(f'{url}{id}', headers=headers)
        if response.status_code == 200:
            data = response.json()
            courses_content.append(create_text(data))
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")

    return courses_content
    


TODO: scrap ou noté les codes des cours qu'on veut scrapé

In [3]:
def find_courses(children):
    all_courses = []
    for child in children:
        if len(child['children']) > 0: find_courses(child['children'])
        else: all_courses.append([course['teachingId'] for course in child['listTeaching']])
    return all_courses

In [4]:
def get_all_courses():
    response = requests.get('https://pgc.unige.ch/main/api/faculties?size=10000&academicalYear=2023', headers=headers)
    facIds = []
    planIds = []
    coursesIds = []
    if response.status_code == 200:
        for faculte in response.json()['_data']:
            facIds.append(faculte['resId'])
            response2 = requests.get(f"https://pgc.unige.ch/main/api/study-plans?academicalYear=2023&facultyId={faculte['resId']}")
            if response2.status_code == 200:
                for study_plan in response2.json()['_data'][:1]:
                    planIds.append(study_plan['resId'])
                    response3 = requests.get(f"https://pgc.unige.ch/main/api/study-plan-nodes/{study_plan['resId']}")
                    if response3.status_code == 200:
                        # print(response3.json()['children'])
                        courses = find_courses(response3.json()['children'])
                        coursesIds.append(courses)
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    return coursesIds

In [5]:
coursesIds = get_all_courses()
print(coursesIds)

[[['2023-11X001', '2023-11X002', '2023-11X006', '2023-11M010', '2023-11M020', '2023-11P090', '2023-11X003', '2023-11X020', '2023-11X005', '2023-11X004', '2023-11P091', '2023-11M005', '2023-11M050', '2023-11X008'], ['2023-12M061', '2023-12M040', '2023-12X009', '2023-12X001', '2023-12X014', '2023-12X005', '2023-12X050', '2023-12X008', '2023-12X003', '2023-12X006', '2023-12X004', '2023-D200025', '2023-12X015'], ['2023-13X001', '2023-13X007', '2023-13X003', '2023-13X009', '2023-13X005', '2023-13X004', '2023-13X012', '2023-13X008', '2023-13X011', '2023-1MEMINFO']], [['2023-213P006', '2023-213P007', '2023-213P009'], ['2023-223A002', '2023-223A003', '2023-223A004', '2023-223A006', '2023-223A007', '2023-223A008', '2023-223A015', '2023-223TP001', '2023-223C100', '2023-223D100', '2023-223M001', '2023-223IP001', '2023-223O001', '2023-223O002', '2023-223S001', '2023-223S002'], ['2023-223A010', '2023-223A011', '2023-223A012', '2023-223A013', '2023-223C300', '2023-223D300', '2023-223A016', '2023-223

In [6]:
flattened_list = [item for sublist1 in coursesIds for sublist2 in sublist1 for item in sublist2]

print(flattened_list)

['2023-11X001', '2023-11X002', '2023-11X006', '2023-11M010', '2023-11M020', '2023-11P090', '2023-11X003', '2023-11X020', '2023-11X005', '2023-11X004', '2023-11P091', '2023-11M005', '2023-11M050', '2023-11X008', '2023-12M061', '2023-12M040', '2023-12X009', '2023-12X001', '2023-12X014', '2023-12X005', '2023-12X050', '2023-12X008', '2023-12X003', '2023-12X006', '2023-12X004', '2023-D200025', '2023-12X015', '2023-13X001', '2023-13X007', '2023-13X003', '2023-13X009', '2023-13X005', '2023-13X004', '2023-13X012', '2023-13X008', '2023-13X011', '2023-1MEMINFO', '2023-213P006', '2023-213P007', '2023-213P009', '2023-223A002', '2023-223A003', '2023-223A004', '2023-223A006', '2023-223A007', '2023-223A008', '2023-223A015', '2023-223TP001', '2023-223C100', '2023-223D100', '2023-223M001', '2023-223IP001', '2023-223O001', '2023-223O002', '2023-223S001', '2023-223S002', '2023-223A010', '2023-223A011', '2023-223A012', '2023-223A013', '2023-223C300', '2023-223D300', '2023-223A016', '2023-223P100', '2023-2

In [7]:
scraped_courses = scrap_courses(flattened_list)

In [13]:
scraped_courses[0]

{'content': "Le nom du cours est Introduction à la programmation des algorithmes. Le code du cours est 11X001. Le cours est dispensé par la Faculté des sciences. Ce cours est de niveau Bachelor / Master. Ce cours vaut 6.0 crédits. L'objectif de ce cours est: Ce cours a pour but d'introduire les concepts fondamentaux de la programmation des ordinateurs et de l'algorithmique. Des algorithmes représentatifs de problèmes classiques sont étudiés.. La description de ce cours est la suivante: Concepts d'algorithmes, notions fondamentales, abstraction, séquences, itérations, récursivité.\r\n\r\n'\tProgrammes et langages de programmation, compilateurs et interpréteurs.\r\n\r\n'\tFondamentaux de la programmation :\r\n\r\n+ modèle de von Neumann, mémoire,\r\n+ types primitifs,\r\n+ tableaux et chaînes de caractères,\r\n+ structures et énumérations,\r\n+ instructions d'affectation et de contrôle,\r\n+ fonctions, récursivité,\r\n+ fonctions anonymes et d'ordres supérieurs.\r\n\r\n'\tPratique de la 

In [22]:
import json

docs_to_save = []
for doc in scraped_courses:
    docs_to_save.append({"content": doc['content'], "metadata": doc['metadata']})
    with open("../pre-processed/cours.jsonl", 'w') as f:
        for item in docs_to_save:
            f.write(json.dumps(item) + "\n")